# Imports

In [3]:
import os, sys, json

# Determine project root (parent of notebooks directory)
current_dir = os.path.abspath(os.getcwd())
# If running from project root, notebooks/ may not be in path; handle both cases
if os.path.basename(current_dir).lower() == "notebooks":
    project_root = os.path.abspath(os.path.join(current_dir, ".."))
else:
    # Assume we launched from project root already
    project_root = current_dir

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

try:
    from networks import SpikingNetwork
except ModuleNotFoundError as e:
    raise RuntimeError(f"Failed to import SpikingNetwork from {src_path}: {e}")
try:
    from connections import *
except ModuleNotFoundError as e:
    raise RuntimeError(f"Failed to import connections from {src_path}: {e}")

# Load config file robustly
config_path = os.path.join(project_root, "configs", "baseline_lif.json")
with open(config_path, "r") as f:
    config = json.load(f)
print("Loaded config from", config_path)

Loaded config from c:\Users\Work\VSCodeProjects\spiking-network-info-transmission\configs\baseline_lif.json


# Create weight matrices & network

In [4]:
net = SpikingNetwork(config)
ff1 = connections.connect_distance(config["n_inputs"], config["n_layer_one"])
ff2 = connections.connect_distance(config["n_layer_one"], config["n_layer_two"])
ff3 = connections.connect_distance(config["n_layer_two"], config["n_outputs"])
rec1 = connections.create_clustered_connection_with_density(**config["clustered_connection_params"])
rec2 = connections.create_clustered_connection_with_density(**config["clustered_connection_params"])
rec3 = connections.create_clustered_connection_with_density(**config["clustered_connection_params"])

add_weights(self, ff1, rec1, ff2, rec2, ff3, rec3)

AssertionError: Must provide either no. of neurons or shape of layer